In [1]:
type Parser a = String -> [(a,String)]

In [2]:
anyChar :: Parser Char
anyChar (c:cs) = [(c, cs)]
anyChar _      = []

In [3]:
anyChar "abcde"
anyChar "hijkl"

[('a',"bcde")]

[('h',"ijkl")]

In [5]:
char :: Char -> Parser Char
char c' (c:cs) | c'==c = [(c,cs)]
char _  _              = []

In [6]:
char 'a' "abcde"
char 'a' "hijkl"

[('a',"bcde")]

[]

In [8]:
oneOf xs (c:cs) | c `elem` xs = [(c,cs)]
oneOf _  _                    = []

In [9]:
oneOf ['a','b','c'] "abcde"
oneOf ['a','b','c'] "bcdef"
oneOf ['a','b','c'] "cdefg"
oneOf ['a','b','c'] "defgh"

[('a',"bcde")]

[('b',"cdef")]

[('c',"defg")]

[]

In [12]:
satisfy :: (Char -> Bool) -> Parser Char
satisfy p (c:cs) | p c = [(c,cs)] 
satisfy _  _           = []

In [13]:
anyChar = satisfy (const True) -- statisfy (\_ -> True)
char x = satisfy (x==)
oneOf xs = satisfy (`elem` xs)

In [14]:
anyChar "abcde"
anyChar "hijkl"

[('a',"bcde")]

[('h',"ijkl")]

In [15]:
char 'a' "abcde"
char 'a' "hijkl"

[('a',"bcde")]

[]

In [17]:
oneOf ['a','b','c'] "abcde"
oneOf ['a','b','c'] "bcdef"
oneOf ['a','b','c'] "cdefg"
oneOf ['a','b','c'] "defgh"

[('a',"bcde")]

[('b',"cdef")]

[('c',"defg")]

[]

In [18]:
import Data.Char
space = satisfy isSpace
alphaNum = satisfy isAlphaNum
letter = satisfy isAlpha
lower = satisfy isLower
upper = satisfy isUpper
digit = satisfy isDigit

In [19]:
digit "abcde"
digit "123abc"

[]

[('1',"23abc")]

아래의 실행예처럼 주어진 문자열과 정확히 일치하는 만큼을 분석하여 처리하는 `string` 함수를 작성하라. (다음 과제의 첫 문제가 될 내용이다)
```haskell
string "abc" "abcdef" == [("abc","def")]
```

In [26]:
string :: String -> Parser String 
string ""     cs                 = [("",cs)]
string (x:xs) (c:cs) | undefined = undefined
string _      _                  = []

----

다음 과제의 두번째 문제는 아래 모든 개수를 다 고려하는 문법분석 함수 many1과 many를 가장 길게 매칭되는 경우만 결과로 하는 것으로 수정하라는 것

In [22]:
-- parse one or more times
many1 :: Parser a -> Parser [a]
many1 p cs = [([x], cs') | (x,cs')<-ps] -- 한번 처리한 결과 x를 리스트로
            ++
             [(x:xs, cs'') | (x,cs')<-ps, (xs,cs'')<-many1 p cs']
           where ps = p cs

-- parse zero or more times
many p cs = [([],cs)] ++ many1 p cs

In [23]:
letter "hello world"

[('h',"ello world")]

In [24]:
many1 letter "hello world"
many1 letter "   hello world"

[("h","ello world"),("he","llo world"),("hel","lo world"),("hell","o world"),("hello"," world")]

[]

In [25]:
many letter "hello world"
many letter "   hello world"

[("","hello world"),("h","ello world"),("he","llo world"),("hel","lo world"),("hell","o world"),("hello"," world")]

[("","   hello world")]

In [27]:
spaces = many space

In [28]:
spaces "hello world"
spaces "   hello world"

[("","hello world")]

[("","   hello world"),(" ","  hello world"),("  "," hello world"),("   ","hello world")]

## 십진수 자연수(양의 정수)를 분석하기

In [31]:
-- 맨 앞에는 0이 아닌 숫자, 그 다음에는 아무 십진수 숫자나

firstDigit = oneOf ['1'..'9'] -- 맨 앞글자를 처리 (딱 1개 글자)
digits = many digit       -- 뒷글자들을 처리 (0개 이상 글자)

In [33]:
firstDigit "123 abc"
firstDigit "023 abc"

[('1',"23 abc")]

[]

In [34]:
digits "123 abc"
digits "023 abc"

[("","123 abc"),("1","23 abc"),("12","3 abc"),("123"," abc")]

[("","023 abc"),("0","23 abc"),("02","3 abc"),("023"," abc")]